In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from skimage import io, measure, color
from scipy import ndimage, stats
import glob
from google.colab import drive
import math

# Mount Google Drive if not already mounted
try:
    drive.mount('/content/drive')
    print("Drive mounted successfully")
except:
    print("Drive already mounted")

# Define paths
input_path = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Cell"
output_path = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3_1.4Pa_18h"

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created output directory: {output_path}")

# Function to calculate morphometric properties for a labeled mask
def calculate_morphometrics(labeled_mask):
    properties = measure.regionprops(labeled_mask)

    # Collect properties for each region
    results = []
    for prop in properties:
        # Skip small regions (possible noise)
        if prop.area < 50:
            continue

        # Calculate orientation angle in degrees
        # Convert from (-pi/2, pi/2) range to (0, 180) range
        orientation = np.degrees(prop.orientation)
        if orientation < 0:
            orientation += 180

        # Calculate alignment with flow direction (left to right = 0 degrees)
        # Values close to 0 or 180 indicate parallel alignment
        # Values close to 90 indicate perpendicular alignment
        flow_alignment = min(orientation, abs(180 - orientation))

        # Calculate aspect ratio (major axis length / minor axis length)
        aspect_ratio = prop.major_axis_length / prop.minor_axis_length if prop.minor_axis_length > 0 else np.nan

        # Calculate circularity (4π × area / perimeter²)
        circularity = 4 * np.pi * prop.area / (prop.perimeter ** 2) if prop.perimeter > 0 else np.nan

        results.append({
            'area': prop.area,
            'perimeter': prop.perimeter,
            'centroid_x': prop.centroid[1],
            'centroid_y': prop.centroid[0],
            'major_axis_length': prop.major_axis_length,
            'minor_axis_length': prop.minor_axis_length,
            'eccentricity': prop.eccentricity,
            'orientation_degrees': orientation,
            'flow_alignment_degrees': flow_alignment,
            'aspect_ratio': aspect_ratio,
            'circularity': circularity,
            'solidity': prop.solidity
        })

    return pd.DataFrame(results)

# Function to read and process an image file
def process_image(file_path):
    # Extract the Pa value from the filename
    if '0Pa' in os.path.basename(file_path):
        pa_value = 0
    elif '1.4Pa' in os.path.basename(file_path):
        pa_value = 1.4
    else:
        pa_value = np.nan

    # Read the image
    try:
        # For TIFF files with multiple color labels
        img = io.imread(file_path)

        # Check if image is a labeled mask or needs to be labeled
        if img.ndim == 3:  # Color image
            # If it's a color-labeled image, convert to labeled mask
            # This assumes each color represents a unique cell
            gray = color.rgb2gray(img)
            unique_labels = np.unique(img.reshape(-1, img.shape[2]), axis=0)
            labeled_mask = np.zeros(gray.shape, dtype=np.int32)

            for i, label in enumerate(unique_labels[1:], 1):  # Skip background (usually black)
                mask = np.all(img == label.reshape(1, 1, -1), axis=2)
                labeled_mask[mask] = i
        else:
            # If it's already a labeled mask, use it directly
            labeled_mask = img

        # Calculate morphometrics
        morphometrics = calculate_morphometrics(labeled_mask)
        morphometrics['pressure_pa'] = pa_value
        morphometrics['image_file'] = os.path.basename(file_path)

        return morphometrics, labeled_mask

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None, None

# Function to visualize alignment with flow direction
def visualize_cell_alignment(labeled_mask, df, output_file):
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # 1. Visualization of labeled cells with orientation
    ax1.imshow(color.label2rgb(labeled_mask, bg_label=0))
    ax1.set_title('Cell Masks with Orientation')

    # Draw orientation lines for each cell
    for _, row in df.iterrows():
        y, x = row['centroid_y'], row['centroid_x']
        angle = row['orientation_degrees']
        length = row['major_axis_length'] / 2

        # Calculate end points of the orientation line
        dx = length * np.cos(np.radians(angle))
        dy = length * np.sin(np.radians(angle))

        # Draw the line
        ax1.plot([x - dx, x + dx], [y - dy, y + dy], 'r-', linewidth=2)

    ax1.set_xlabel('Flow Direction →')

    # 2. Histogram of alignment angles
    ax2.hist(df['flow_alignment_degrees'], bins=18, range=(0, 90), alpha=0.7)
    ax2.set_xlabel('Alignment Angle (degrees)')
    ax2.set_ylabel('Number of Cells')
    ax2.set_title('Cell Alignment with Flow Direction')
    ax2.axvline(x=45, color='r', linestyle='--', label='Random Orientation')
    ax2.set_xlim(0, 90)
    ax2.legend()

    # Add a text box with statistics
    mean_alignment = df['flow_alignment_degrees'].mean()
    median_alignment = df['flow_alignment_degrees'].median()
    alignment_std = df['flow_alignment_degrees'].std()

    # Classify alignment trend
    if mean_alignment < 30:
        trend = "Mostly parallel to flow"
    elif mean_alignment > 60:
        trend = "Mostly perpendicular to flow"
    else:
        trend = "Mixed orientation"

    stats_text = (
        f"Mean Alignment: {mean_alignment:.2f}°\n"
        f"Median Alignment: {median_alignment:.2f}°\n"
        f"Standard Deviation: {alignment_std:.2f}°\n"
        f"Trend: {trend}"
    )

    # Add text box
    ax2.text(0.05, 0.95, stats_text, transform=ax2.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.close()

    return mean_alignment, trend

# Function to generate comparison plots between 0Pa and 1.4Pa
def generate_comparison_plots(df_0pa, df_1_4pa, output_path):
    # Combine dataframes for comparison
    df_0pa['pressure_group'] = '0 Pa'
    df_1_4pa['pressure_group'] = '1.4 Pa'
    combined_df = pd.concat([df_0pa, df_1_4pa])

    # List of properties to compare
    properties = ['area', 'perimeter', 'major_axis_length', 'minor_axis_length',
                 'eccentricity', 'aspect_ratio', 'circularity', 'solidity', 'flow_alignment_degrees']

    # Loop through properties and create comparison plots
    for prop in properties:
        plt.figure(figsize=(10, 6))

        # Create violin plots for comparison
        ax = sns.violinplot(x='pressure_group', y=prop, data=combined_df, inner='box')

        # Add statistical annotation
        mean_0pa = df_0pa[prop].mean()
        mean_1_4pa = df_1_4pa[prop].mean()
        percent_change = ((mean_1_4pa - mean_0pa) / mean_0pa) * 100

        # Statistical test (t-test)
        t_stat, p_val = stats.ttest_ind(df_0pa[prop].dropna(), df_1_4pa[prop].dropna(), equal_var=False)

        # Add information to plot
        plt.title(f'Comparison of {prop.replace("_", " ").title()}')

        # Add text with statistics
        stat_text = (
            f"Mean (0 Pa): {mean_0pa:.2f}\n"
            f"Mean (1.4 Pa): {mean_1_4pa:.2f}\n"
            f"Change: {percent_change:.2f}%\n"
            f"p-value: {p_val:.4f}"
        )

        # Determine significance level
        if p_val < 0.001:
            sig = '***'
        elif p_val < 0.01:
            sig = '**'
        elif p_val < 0.05:
            sig = '*'
        else:
            sig = 'ns'

        # Add significance bar
        y_max = combined_df[prop].max() * 1.1
        x1, x2 = 0, 1
        plt.plot([x1, x1, x2, x2], [y_max, y_max*1.05, y_max*1.05, y_max], lw=1.5, c='black')
        plt.text((x1+x2)*.5, y_max*1.07, sig, ha='center', va='bottom', color='black')

        # Add text box with statistics
        plt.text(0.05, 0.95, stat_text, transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        # Save the figure
        output_file = os.path.join(output_path, f'comparison_{prop}.png')
        plt.tight_layout()
        plt.savefig(output_file, dpi=300)
        plt.close()

    # Create rose plots for orientation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': 'polar'})

    # 0 Pa rose plot
    angle_bins = np.linspace(0, np.pi, 19)  # 0 to 180 degrees in 10-degree bins
    # Convert angles to 0-180 range for histogram
    angles_0pa = df_0pa['orientation_degrees'].values
    hist_0pa, _ = np.histogram(np.radians(angles_0pa), bins=angle_bins)
    width = angle_bins[1] - angle_bins[0]
    ax1.bar(angle_bins[:-1], hist_0pa, width=width, alpha=0.7)
    ax1.set_title('Cell Orientation at 0 Pa')
    ax1.set_theta_zero_location('E')  # East = 0 degrees (flow direction)
    ax1.set_theta_direction(-1)
    ax1.set_thetamin(0)
    ax1.set_thetamax(180)

    # 1.4 Pa rose plot
    angles_1_4pa = df_1_4pa['orientation_degrees'].values
    hist_1_4pa, _ = np.histogram(np.radians(angles_1_4pa), bins=angle_bins)
    ax2.bar(angle_bins[:-1], hist_1_4pa, width=width, alpha=0.7)
    ax2.set_title('Cell Orientation at 1.4 Pa')
    ax2.set_theta_zero_location('E')  # East = 0 degrees (flow direction)
    ax2.set_theta_direction(-1)
    ax2.set_thetamin(0)
    ax2.set_thetamax(180)

    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'orientation_rose_plots.png'), dpi=300)
    plt.close()

    # Create summary table
    summary_data = []
    for prop in properties:
        mean_0pa = df_0pa[prop].mean()
        mean_1_4pa = df_1_4pa[prop].mean()
        median_0pa = df_0pa[prop].median()
        median_1_4pa = df_1_4pa[prop].median()
        std_0pa = df_0pa[prop].std()
        std_1_4pa = df_1_4pa[prop].std()

        t_stat, p_val = stats.ttest_ind(df_0pa[prop].dropna(), df_1_4pa[prop].dropna(), equal_var=False)

        summary_data.append({
            'Property': prop.replace('_', ' ').title(),
            'Mean_0Pa': mean_0pa,
            'Mean_1.4Pa': mean_1_4pa,
            'Median_0Pa': median_0pa,
            'Median_1.4Pa': median_1_4pa,
            'StdDev_0Pa': std_0pa,
            'StdDev_1.4Pa': std_1_4pa,
            'Percent_Change': ((mean_1_4pa - mean_0pa) / mean_0pa) * 100,
            'p_value': p_val,
            'Significant': p_val < 0.05
        })

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(os.path.join(output_path, 'morphology_summary.csv'), index=False)

    return summary_df

# Main execution
def main():
    # Find all mask files
    mask_files = glob.glob(os.path.join(input_path, "*.tif")) + glob.glob(os.path.join(input_path, "*.tiff"))

    if not mask_files:
        print(f"No mask files found in {input_path}")
        return

    print(f"Found {len(mask_files)} mask files")

    # Separate files by pressure
    files_0pa = [f for f in mask_files if '0Pa' in os.path.basename(f)]
    files_1_4pa = [f for f in mask_files if '1.4Pa' in os.path.basename(f)]

    print(f"0 Pa files: {len(files_0pa)}")
    print(f"1.4 Pa files: {len(files_1_4pa)}")

    # Process 0 Pa files
    results_0pa = []
    for file_path in files_0pa:
        print(f"Processing {os.path.basename(file_path)}...")
        morphometrics, labeled_mask = process_image(file_path)

        if morphometrics is not None and not morphometrics.empty:
            results_0pa.append(morphometrics)

            # Visualize and save individual file results
            base_name = os.path.splitext(os.path.basename(file_path))[0]
            output_file = os.path.join(output_path, f"{base_name}_analysis.png")
            mean_alignment, trend = visualize_cell_alignment(labeled_mask, morphometrics, output_file)
            print(f"  - Mean alignment: {mean_alignment:.2f}° ({trend})")

    # Process 1.4 Pa files
    results_1_4pa = []
    for file_path in files_1_4pa:
        print(f"Processing {os.path.basename(file_path)}...")
        morphometrics, labeled_mask = process_image(file_path)

        if morphometrics is not None and not morphometrics.empty:
            results_1_4pa.append(morphometrics)

            # Visualize and save individual file results
            base_name = os.path.splitext(os.path.basename(file_path))[0]
            output_file = os.path.join(output_path, f"{base_name}_analysis.png")
            mean_alignment, trend = visualize_cell_alignment(labeled_mask, morphometrics, output_file)
            print(f"  - Mean alignment: {mean_alignment:.2f}° ({trend})")

    # Combine results
    if results_0pa and results_1_4pa:
        combined_0pa = pd.concat(results_0pa)
        combined_1_4pa = pd.concat(results_1_4pa)

        # Save combined data
        combined_0pa.to_csv(os.path.join(output_path, "0Pa_morphometrics.csv"), index=False)
        combined_1_4pa.to_csv(os.path.join(output_path, "1.4Pa_morphometrics.csv"), index=False)

        # Generate comparison plots and summary
        summary_df = generate_comparison_plots(combined_0pa, combined_1_4pa, output_path)

        # Print summary of findings
        print("\nSummary of Findings:")
        print("-" * 80)
        for _, row in summary_df.iterrows():
            sig_mark = '*' if row['Significant'] else 'ns'
            print(f"{row['Property']}: {row['Percent_Change']:.2f}% change ({sig_mark})")

        # Create a summary figure with key findings
        plt.figure(figsize=(12, 8))
        plt.subplot(2, 2, 1)

        # Plot area comparison
        area_data = summary_df[summary_df['Property'] == 'Area']
        plt.bar(['0 Pa', '1.4 Pa'], [area_data['Mean_0Pa'].values[0], area_data['Mean_1.4Pa'].values[0]])
        plt.title('Cell Area')
        plt.ylabel('Area (pixels)')

        # Plot alignment comparison
        plt.subplot(2, 2, 2)
        align_data = summary_df[summary_df['Property'] == 'Flow Alignment Degrees']
        plt.bar(['0 Pa', '1.4 Pa'], [align_data['Mean_0Pa'].values[0], align_data['Mean_1.4Pa'].values[0]])
        plt.title('Flow Alignment')
        plt.ylabel('Degrees')

        # Plot aspect ratio comparison
        plt.subplot(2, 2, 3)
        aspect_data = summary_df[summary_df['Property'] == 'Aspect Ratio']
        plt.bar(['0 Pa', '1.4 Pa'], [aspect_data['Mean_0Pa'].values[0], aspect_data['Mean_1.4Pa'].values[0]])
        plt.title('Aspect Ratio')

        # Plot circularity comparison
        plt.subplot(2, 2, 4)
        circ_data = summary_df[summary_df['Property'] == 'Circularity']
        plt.bar(['0 Pa', '1.4 Pa'], [circ_data['Mean_0Pa'].values[0], circ_data['Mean_1.4Pa'].values[0]])
        plt.title('Circularity')

        plt.tight_layout()
        plt.savefig(os.path.join(output_path, 'key_findings_summary.png'), dpi=300)
        plt.close()
    else:
        print("Error: No valid data for one or both pressure conditions")

    print("\nAnalysis complete! Results saved to:", output_path)

# Run the main function
if __name__ == "__main__":
    main()

Mounted at /content/drive
Drive mounted successfully
Found 8 mask files
0 Pa files: 3
1.4 Pa files: 5
Processing denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_cell_mask.tif...
  - Mean alignment: 46.08° (Mixed orientation)
Processing denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_cell_mask.tif...
  - Mean alignment: 48.33° (Mixed orientation)
Processing denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq003_cell_mask.tif...
  - Mean alignment: 44.99° (Mixed orientation)
Processing denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq001_cell_mask.tif...
  - Mean alignment: 53.51° (Mixed orientation)
Processing denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq003_cell_mask.tif...
  - Mean alignment: 53.32° (Mixed orientation)
Processing denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq002_cell_mask.tif...
  - Mean alignment: 62.14° (Mostly perpendicular to flow)
Processing denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq004_cell_mask.tif...
  - Mean alignment: 41.19° (Mixed orientation)
Processing denoised_1.4Pa_U_05mar19_20x_L2R_Flat_s